#### $step1.$导包

In [53]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import random
from copy import deepcopy as dcp
import heapq 

---
#### $step2.$导入数据

In [54]:
Train = pd.read_csv('/kaggle/input/ml-100k/u.dataTrain', header = None, sep = '\t')
print(Train)
train = Train.values

         0    1  2          3
0        0  167  5  874965478
1        0  171  5  874965478
2        0  164  5  874965518
3        0  155  4  874965556
4        0  195  5  874965677
...    ...  ... ..        ...
80362  942  414  1  888640027
80363  942  719  1  888640048
80364  942   79  2  888640048
80365  942   52  3  888640067
80366  942  216  3  888640067

[80367 rows x 4 columns]


In [55]:
Test = pd.read_csv('/kaggle/input/ml-100k/u.dataTest', header = None, sep = '\t')
print(Test)
test = Test.values

         0    1  2          3
0        0  200  3  878542960
1        0  207  5  878542960
2        0  115  3  878542960
3        0   57  4  878542960
4        0    2  4  878542960
...    ...  ... ..        ...
19544  942  448  1  888693158
19545  942  228  2  888693158
19546  942  229  1  888693158
19547  942  227  3  888693158
19548  942  233  3  888693184

[19549 rows x 4 columns]


In [56]:
Matrix = pd.read_csv('/kaggle/input/ml-100k/u.dataTrainmat', header = None, sep = '\t')
print(Matrix)
matrix = Matrix.values

     0     1     2     3     4     5     6     7     8     9     ...  1671  \
0     5.0   3.0   0.0   3.0   0.0   0.0   4.0   1.0   0.0   3.0  ...   0.0   
1     4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0  ...   0.0   
2     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4     4.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
938   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   5.0   0.0  ...   0.0   
939   0.0   0.0   0.0   0.0   0.0   0.0   4.0   5.0   3.0   0.0  ...   0.0   
940   0.0   0.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0  ...   0.0   
941   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
942   0.0   5.0   0.0   0.0   0.0   0.0   0.0   0.0   3.0   0.0  ...   0.0   

     1672  1673  1674  1675  1676  1677  1678  1679  1680  
0  

---
#### $step3.$ KNN

In [57]:
class _KNN():
    # written by pipedream
    # matrix_version
    def __init__(self):
        self.has_cos = False
        self.has_predict_matrix = False
        self.kvalue_top_k = -1
        self.MAE = -1
        self.RMSD = -1
        self.cos = []
        self.topk = []
        self.matrix = []
        self.avg_ru = []
        self.predict_matrix = []
        
    def CosineSimilar(self, _ratings):
        for i in tqdm(range(1)):
            ratings = np.array(_ratings)
            H = ratings.copy()
            R = ratings.copy()
            H[H > 0] = 1

            up = np.dot(R, R.T)
            norm1 = np.dot(np.multiply(R, R), H.T)
            norm2 = np.dot(H, np.multiply(R, R).T)
            down = np.sqrt(np.multiply(norm1, norm2))
            down[down == 0] = 1

            sim = up / down
            np.fill_diagonal(sim, 0)
        return sim

    # input matrix: persons-items
    def fit(self, _matrix, KNN_k):
        
        matrix = dcp(np.array(_matrix))
        person_count = len(matrix)
        item_count = len(matrix[0])
        diff = 0
        
        #---- cal cos_matrix ----
        if self.has_cos == False or self.matrix.all() != matrix.all():
            
            avg_ru = []
            for i in range(person_count):
                u, d = 0, 0
                for j in range(item_count):
                    if matrix[i][j] != 0:
                        u += matrix[i][j]
                        d += 1
                avg_ru.append(u / d if d != 0 else 0)
            
            self.has_cos = True
            self.avg_ru = dcp(avg_ru)
            self.matrix = dcp(matrix)
            self.cos = dcp(self.CosineSimilar(matrix))
            
            topk = [] # person -> topk_neighbours : [[cos, idx], [cos, idx]]
            
            for i in tqdm(range(person_count)):
                def cal(x):
                    return self.cos[i][x]
                per_topk = [tk for tk in range(person_count) if self.cos[i][tk] > 0]
                per_topk.sort(key = cal)
                topk.append(per_topk)

        
            self.topk = dcp(topk)
            diff += 1

    
        #---- cal predict_matrix ----
        if self.has_predict_matrix == False or self.kvalue_top_k != KNN_k or diff > 0:

            self.predict_matrix = dcp(matrix)
            self.predict_matrix = self.predict_matrix.astype(float)
            
            pos = np.where(matrix == 0)
            x, y = pos[0], pos[1]
            for idx in tqdm(range(len(x))):
                i, j, up, dw, cnt = x[idx], y[idx], 0, 0, 0
                for k in self.topk[i]:
                    if matrix[k][j] == 0: 
                        continue
                    cnt += 1
                    up += (matrix[k][j] - self.avg_ru[k]) * self.cos[i][k]
                    dw += self.cos[i][k]
                    if cnt >= KNN_k:
                        break
                            
                self.predict_matrix[i][j] = self.avg_ru[i] + ((up / dw) if dw != 0 else random.uniform(0, 5))

            self.has_predict_matrix = True
            self.kvalue_top_k = KNN_k

    
    def predict(self, Test):
        
        _MAE = 0
        _RMSD = 0
        test_person_count = len(test)
        for data in Test:
            person, item, rating, ti = data
            _MAE += abs(self.predict_matrix[person][item] - rating)
            _RMSD += (self.predict_matrix[person][item] - rating) * (self.predict_matrix[person][item] - rating)
            
        self.MAE = _MAE / test_person_count
        self.RMSD = np.sqrt(_RMSD / test_person_count)
    
    def score(self):
        
        if self.MAE == -1:
            print('No-Test-Perdict')
        else:
            print('MAE: ', self.MAE)
            print('RMSD: ', self.RMSD)

In [58]:
knn = _KNN()
knn.fit(matrix, 2)

100%|██████████| 1504816/1504816 [03:03<00:00, 8182.13it/s] 


In [59]:
knn.predict(test)
knn.score()

MAE:  1.1145303710922603
RMSD:  1.3955974255720653


In [60]:
knn.fit(matrix, 3)
knn.predict(test)
knn.score()

100%|██████████| 1504816/1504816 [03:58<00:00, 6304.69it/s]


MAE:  1.0220422704246441
RMSD:  1.2809618983482334


In [61]:
knn.fit(matrix, 5)
knn.predict(test)
knn.score()

100%|██████████| 1504816/1504816 [05:07<00:00, 4899.54it/s]


MAE:  0.9455281700756469
RMSD:  1.187782673614962


In [62]:
knn.fit(matrix, 10)
knn.predict(test)
knn.score()

100%|██████████| 1504816/1504816 [06:46<00:00, 3704.85it/s]

MAE:  0.8845006542160077
RMSD:  1.1101178544389145


In [63]:
knn.fit(matrix, 20)
knn.predict(test)
knn.score()

100%|██████████| 1504816/1504816 [08:22<00:00, 2992.12it/s]

MAE:  0.8475480016405725
RMSD:  1.0648317653016772


In [64]:
mar = [
    [5, 1, 5, 0, 4, 2],
    [5, 0, 5, 1, 4, 1],
    [1, 4, 0, 0, 1, 5],
    [0, 0, 4, 1, 5, 0],
    [4, 2, 4, 0, 5, 1]
]
n = 5
m = 6

----
#### version1: for循环版本KNN（太慢了留作纪念）

In [65]:
class _KNN_by_for():
    # written by pipedream
    # for循环版本
    def __init__(self):
        self.has_cos = False
        self.has_predict_matrix = False
        self.kvalue_top_k = -1
        self.MAE = -1
        self.RMSD = -1
        self.cos = []
        self.topk = []
        self.matrix = []
        self.avg_ru = []
        self.predict_matrix = []
        
    # input matrix: persons-items
    def fit(self, _matrix, KNN_k):
        
        matrix = dcp(np.array(_matrix))
        person_count = len(matrix)
        item_count = len(matrix[0])
        diff = 0
        
        #---- cal cos_matrix ----
        if self.has_cos == False or self.matrix.all() != matrix.all():
            # ----O(N * N * log(N))---- 
            
            cos = []
            for i in tqdm(range(person_count)):
                cos_person = []
                for j in range(person_count):
                    if i == j :
                        cos_person.append(0)
                        continue
                    # cal cos
                    xu, xv, xuxv = 0, 0, 0
                    for k in range(item_count):
                        if matrix[i][k] != 0 and matrix[j][k] != 0:
                            xu += matrix[i][k] * matrix[i][k]
                            xv += matrix[j][k] * matrix[j][k]
                            xuxv += matrix[i][k] * matrix[j][k]
                    xu, xv = np.sqrt(xu), np.sqrt(xv)
                    _cos = xuxv / (xu * xv) if xu * xv != 0 else 0
                    cos_person.append(_cos)
                cos.append(cos_person)
            
            
            avg_ru = []
            for i in range(person_count):
                u, d = 0, 0
                for j in range(item_count):
                    if matrix[i][j] != 0:
                        u += matrix[i][j]
                        d += 1
                avg_ru.append(u / d if d != 0 else 0)
            
            self.has_cos = True
            self.avg_ru = dcp(avg_ru)
            self.matrix = dcp(matrix)
            self.cos = dcp(cos)
            diff += 1

        
        #---- cal topk ----
        if self.kvalue_top_k != KNN_k or diff > 0:
            # ----O(N * N * K)----
            topk = [] # person -> topk_neighbours : [[cos, idx], [cos, idx]]

            for i in tqdm(range(person_count)):
                loop_k = [[1e18, -1]]
                for j in range(person_count):
                    if i == j or self.cos[i][j] == 0:
                        continue
                        
                    # input topk
                    for idx in range(len(loop_k)):
                        
                        if loop_k[idx][0] > self.cos[i][j]:
                            loop_k.insert(idx, [self.cos[i][j], j])
                            break
                    if len(loop_k) > KNN_k + 1:
                        del loop_k[0]

                del loop_k[-1]
                topk.append(loop_k)
                
            self.topk = dcp(topk)
            self.kvalue_top_k = KNN_k
            diff += 1

    
        #---- cal predict_matrix ----
        if self.has_predict_matrix == False or diff > 0:
            # ----O(N * M * K)----
            self.predict_matrix = dcp(_matrix)
            
            for i in tqdm(range(person_count)):
                for j in range(item_count):
                    if matrix[i][j] == 0 :
                        if len(topk[i]) == 0:
                            continue
                        up, dw = 0, 0
                        for tk in topk[i]:
                            _cos = tk[0]
                            k = tk[1]
                            if matrix[k][j] != 0:
                                up += (matrix[k][j] - self.avg_ru[k]) * _cos
                                dw += _cos
                        self.predict_matrix[i][j] = self.avg_ru[i] + (up / dw if dw != 0 else 0)
                        
            self.has_predict_matrix = True
    
    
    def predict(self, Test):
        
        _MAE = 0
        _RMSD = 0
        test_person_count = len(test)
        for data in Test:
            person, item, rating, ti = data
            _MAE += abs(self.predict_matrix[person][item] - rating)
            _RMSD += (self.predict_matrix[person][item] - rating) * (self.predict_matrix[person][item] - rating)
            
        self.MAE = _MAE / test_person_count
        self.RMSD = np.sqrt(_RMSD / test_person_count)
    
    def score(self):
        
        if self.MAE == -1:
            print('No-Test-Perdict')
        else:
            print('MAE: ', self.MAE)
            print('RMSD: ', self.RMSD)
            